In [1]:
import sys
sys.path.append('/workspace/src/')

In [ ]:
from dotenv import dotenv_values, load_dotenv
from umbrela.gpt_judge import GPTJudge
import pickle

import json
from tqdm import tqdm

from preprocessing.utils import parallel_process_topics

db_vals = dotenv_values("/workspace/src/.env")
load_dotenv()

In [ ]:
#MODEL_NAME="gpt-4o-mini"
MODEL_NAME="gpt-4o-2024-11-20"
#RETRIEVER="cosine"
RETRIEVER="bm25"

In [ ]:
judge_gpt = GPTJudge(qrel="test_qrels", prompt_type="bing", engine=MODEL_NAME)
topics = pickle.load(open("/workspace/src/data/topics.pkl", "rb"))

In [4]:
pooling_results_bm25 = pickle.load(open(f"/workspace/src/data/pooling_results_{RETRIEVER}.pkl", "rb"))

In [ ]:
all_judgments =parallel_process_topics(topics, pooling_results_bm25, judge_gpt, max_workers=20)

In [ ]:
with open(f'/workspace/src/data/qrels_bm25_{MODEL_NAME}.json', 'w', encoding='utf-8') as json_file:
    json.dump(all_judgments, json_file, ensure_ascii=False, indent=4)

print("Auto qrels saved")